# Truncated potentials for long-range electrostatics

_Björn Stenqvist, Vidar Aspelin and Mikael Lund, Div. of Theoretical Chemistry, Lund University, Sweden, 2019_

In this notebook we test the validity of a variety of Wolf-inspired potentials and compare with a default Ewald summation on a simple box of water.

Article: "Generalized Moment Cancellation for Long-Range Electrostatics"

Install prerequisites using `conda`:
```bash
$ conda config --add channels omnia
$ conda install -c omnia openmm mdtraj packmol
```

To test the Openmm installation, run

```bash
python -m simtk.testInstallation
```

Also, Gromacs analysis tools (`gmx`) must be installed for parts of the analysis.

In [1]:
%matplotlib inline
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import matplotlib, matplotlib.pyplot as plt
import pandas as pd
from io import StringIO
import numpy as np, os
from matplotlib.font_manager import FontProperties
matplotlib.rcParams.update({'font.size': 12})

# simulation parameters

nsteps     = 1000          # number of MD steps each of 2 fs
nwater     = 2000           # number of water molecules
cutoff     = 0.96*nanometers # pair potential cutoff (0.96, 1.28, 1.6)
platform   = 'OpenCL'         # 'OpenCL', 'CUDA', 'CPU'

rho        = 216/18.6206**3 # density from gromacs spc216.gro
boxlen     = (nwater/rho)**(1/3)
volume     = (boxlen*angstrom)**3

print('volume     = ', volume)
print('box length = ', boxlen*angstrom)
print('half box   = ', boxlen*angstrom/2)

try:
    workdir
except NameError:
    workdir=%pwd
    
%cd $workdir

tempdir = 'cutoff'+str(cutoff).replace(' ', '')
if not os.path.isdir(tempdir):
    %mkdir $tempdir
    
fontSizeLabel=10
fontSizeLegend=8
fontSizeLegend2=6

plt.rcParams.update(
    {
        'font.size': fontSizeLegend2,
        'figure.figsize': [3.513475, 2.713475],
        'xtick.labelsize':fontSizeLegend2,
        'ytick.labelsize':fontSizeLegend2,
        'legend.fontsize':fontSizeLegend
    }
)

volume     =  59780.185333609355 A**3
box length =  39.10080983612078 A
half box   =  19.55040491806039 A
/Users/mikael/github/SI-qpotential/bulk


In [2]:
pf = Platform.getPlatformByName( 'OpenCL' ) # CPU, OpenCL
pf.getSpeed()

50.0

### Create initial box of water

First create a single water molecule pdb file, then use this to fill up a predefined box using the command line tool `packmol`. In this example, $N$ and $L$ are taken form the Gromacs `spc216.gro` file.

In [ ]:
%%writefile hoh.pdb
CRYST1   30.000   30.000   30.000  90.00  90.00  90.00 P 1           1
ATOM      1  OW  HOH A   1      27.552  11.051   7.172  1.00  0.00          O
ATOM      2  HW1 HOH A   1      27.900  10.721   8.050  1.00  0.00          H
ATOM      3  HW2 HOH A   1      26.606  11.355   7.281  1.00  0.00          H 
END

In [ ]:
# write input file for packmol
PACKMOL_INPUT = """ 
tolerance %f
filetype pdb
output %s

# hoh will be put in a box
# defined by the minimum coordinates x, y and z = 0. 0. 0. and maximum
# coordinates box_size box_size box_size That is, they will be put in a cube of side
# box_size (the keyword "inside cube 0. 0. 0. box_size") could be used as well.

structure %s
  number %d
  inside box 0. 0. 0. %f %f %f 
  add_box_sides 0.0
end structure
""" % (2.,'water.pdb','hoh.pdb', nwater, boxlen, boxlen, boxlen)

!echo '$PACKMOL_INPUT' > packmol_input.txt
!/usr/local/bin/packmol < packmol_input.txt > /dev/null

### Create OpenMM `System` classes for a variety of long-range correction schemes

Here the idea is to create all the different setups and add then to the `systemlist` dictionary. Ewald summation (and PME) can be setup using the default `NonbondedForce` class, while `CustomNonbondedForce` must be used for custom pair potentials as used in Wolf, Stenqvist etc.

For more information on how to control force objects and simulation, check,

* [OpenMM python API](http://docs.openmm.org).
* Lennard-Jones + Ewald at http://docs.openmm.org/7.0.0/userguide/theory.html#nonbondedforce
* Long range correction beyond cut-off: http://docs.openmm.org/7.0.0/userguide/theory.html#customnonbondedforce

In [ ]:
%cd $workdir
elec_to_kJmol = (
    constants.elementary_charge**2 *
    AVOGADRO_CONSTANT_NA / (4*np.pi*1.0*8.854187817e-12
                            * constants.farad/constants.meter)).value_in_unit(kilojoule_per_mole*nanometer)

def findForce(system, forcetype, add=True):
    """ Finds a specific force in the system force list - added if not found."""
    for force in system.getForces():
        if isinstance(force, forcetype):
            return force
    if add==True:
        system.addForce(forcetype())
        return findForce(system, forcetype)
    return None

def make_qpotential_system(topology, Rc=0.9*nanometers, moments=1000):
    ''' return a q-potential system (TO '''
    
    ff = ForceField('spce-custom.xml') # this will create a CustomNonbondedForce
    system = ff.createSystem(
        topology, nonbondedMethod=CutoffPeriodic,
        nonbondedCutoff=Rc, constraints=HBonds, rigidWater=True)
    
    def qPochhammerSymbol( Rc, moments ):
        if isinstance(Rc, Quantity):
            Rc = Rc / nanometer # strip unit
        qP = 1.0
        r = np.linspace(0, Rc, 5000)
        for i in range( moments ):
            qP *= (1 - (r/Rc)**(i+1) )
        return qP
 
    qP = Continuous1DFunction( qPochhammerSymbol(Rc, moments), 0*nanometers, Rc)

    nonbonded = findForce(system, CustomNonbondedForce)
    nonbonded.addTabulatedFunction( 'qP', qP )         # 'qP(r)' can now be used in energy function
    nonbonded.addGlobalParameter( 'f', elec_to_kJmol ) # convert to kJ/mol
    
    nonbonded.setEnergyFunction(
        'f * charge1 * charge2 * qP(r)/r' \
        ' + 4 * epsilon * ( (sigma/r)^12 - (sigma/r)^6 )' \
        ' ; sigma = 0.5 * ( sigma1+sigma2 ); epsilon = sqrt( epsilon1*epsilon2 )'
    )

    print('qpot')
    print('    periodic boundaries:  ', nonbonded.usesPeriodicBoundaryConditions())
    print('    switching function:   ', nonbonded.getUseSwitchingFunction())
    print('    long-range correction:', nonbonded.getUseLongRangeCorrection())
    print('    cutoff distance:      ', nonbonded.getCutoffDistance())
    print('    energy function:      ', nonbonded.getEnergyFunction())
    
    return system

def make_SP3_system(topology, Rc=0.9*nanometers):
    ''' return a SP3 system '''
    
    ff = ForceField('spce-custom.xml') # this will create a CustomNonbondedForce
    system = ff.createSystem(
        topology, nonbondedMethod=CutoffPeriodic,
        nonbondedCutoff=Rc, constraints=HBonds, rigidWater=True)
    
    nonbonded = findForce(system, CustomNonbondedForce)
    nonbonded.addGlobalParameter( 'Rc', Rc )           # 'Rc' can now be used in energy function
    nonbonded.addGlobalParameter( 'f', elec_to_kJmol ) # 'lB' bjerrum length in nm and kJ/mol
    
    nonbonded.setEnergyFunction(
        'f * charge1 * charge2 * 1/r * ( 1 - 1.75*r/Rc + 5.25*(r/Rc)^5 - 7*(r/Rc)^6 + 2.5*(r/Rc)^7 )' \
        ' + 4 * epsilon * ( (sigma/r)^12 - (sigma/r)^6 )' \
        ' ; sigma = 0.5 * (sigma1+sigma2); epsilon = sqrt( epsilon1*epsilon2 )'
    )

    print('SP3')
    print('    periodic boundaries:  ', nonbonded.usesPeriodicBoundaryConditions())
    print('    switching function:   ', nonbonded.getUseSwitchingFunction())
    print('    long-range correction:', nonbonded.getUseLongRangeCorrection())
    print('    cutoff distance:      ', nonbonded.getCutoffDistance())
    print('    energy function:      ', nonbonded.getEnergyFunction())
    
    return system

def make_SP1_system(topology, Rc=0.9*nanometers):
    ''' return a SP1 system '''
    
    ff = ForceField('spce-custom.xml') # this will create a CustomNonbondedForce
    system = ff.createSystem(
        topology, nonbondedMethod=CutoffPeriodic,
        nonbondedCutoff=Rc, constraints=HBonds, rigidWater=True)

    nonbonded = findForce(system, CustomNonbondedForce)
    nonbonded.addGlobalParameter( 'Rc', Rc )           # 'Rc' can now be used in energy function
    nonbonded.addGlobalParameter( 'f', elec_to_kJmol ) # 'lB' bjerrum length in nm and kJ/mol
    
    nonbonded.setEnergyFunction(
        'f * charge1 * charge2 * ( 1/r - 1/Rc + 1/Rc^2 * (r-Rc) )' \
        ' + 4 * epsilon * ( (sigma/r)^12 - (sigma/r)^6 )' \
        ' ; sigma = 0.5 * ( sigma1+sigma2 ); epsilon = sqrt( epsilon1 * epsilon2 )'
    )

    print('SP1')
    print('    periodic boundaries:  ', nonbonded.usesPeriodicBoundaryConditions())
    print('    switching function:   ', nonbonded.getUseSwitchingFunction())
    print('    long-range correction:', nonbonded.getUseLongRangeCorrection())
    print('    cutoff distance:      ', nonbonded.getCutoffDistance())
    print('    energy function:      ', nonbonded.getEnergyFunction())
    
    return system

def make_ewald_system(topology, Rc=0.9*nanometers, method=Ewald, ljcorr=False):
    ''' returns an Ewald system '''
    ff = ForceField('spce.xml') # this will create a NonbondedForce
    
    system = ff.createSystem(
        topology, nonbondedMethod=method,
        nonbondedCutoff=Rc, constraints=HBonds, rigidWater=True)
    
    nonbonded = findForce(system, NonbondedForce)
    nonbonded.setUseDispersionCorrection( ljcorr )

    print('ewald')
    print('    err. tolerance:       ', nonbonded.getEwaldErrorTolerance())
    print('    LJ switching function:', nonbonded.getUseSwitchingFunction())
    print('    LJ correction:        ', nonbonded.getUseDispersionCorrection())
    print('    cutoff distance:      ', nonbonded.getCutoffDistance())
 
    return system

pdb = PDBFile('water.pdb')

# here we define our list of system incl. plotting properties. All files from simulation
# and analysis will be prefixed with 'qpot.', 'ewald.', etc.

from collections import OrderedDict

runSystems=True
nptOn=False

systemlist = OrderedDict([
    ('ewald', {
        'system': make_ewald_system( pdb.topology, Rc=cutoff ), 'npt': nptOn,
        'run': runSystems, 'color': 'black', 'label': 'No', 'ls':'-', 'alpha':1, 'plot':True, 'linewidth':2
    }),
    ('SP1', {
        'system': make_SP1_system( pdb.topology, Rc=cutoff ), 'npt': nptOn,
        'run': runSystems, 'color': 'magenta', 'label': 'SP1', 'ls':':', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('SP3', {
        'system': make_SP3_system( pdb.topology, Rc=cutoff ), 'npt': nptOn,
        'run': runSystems, 'color': 'magenta', 'label': 'SP3', 'ls':'-.', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot1', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=1 ), 'npt': nptOn,
        'run': runSystems, 'color': 'red', 'label': '$q$ ($n=1$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot2', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=2 ), 'npt': nptOn,
        'run': runSystems, 'color': 'orange', 'label': '$q$ ($n=2$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot3', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=3 ), 'npt': nptOn,
        'run': runSystems, 'color': 'yellow', 'label': '$q$ ($n=3$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot4', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=4 ), 'npt': nptOn,
        'run': runSystems, 'color': 'green', 'label': '$q$ ($n=4$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot5', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=5 ), 'npt': nptOn,
        'run': runSystems, 'color': 'cyan', 'label': '$q$ ($n=5$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot6', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=6 ), 'npt': nptOn,
        'run': runSystems, 'color': 'blue', 'label': '$q$ ($n=6$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot7', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=7 ), 'npt': nptOn,
        'run': runSystems, 'color': 'purple', 'label': '$q$ ($n=7$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot8', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff, moments=8 ), 'npt': nptOn,
        'run': runSystems, 'color': 'grey', 'label': '$q$ ($n=8$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    }),
    ('qpot', {
        'system': make_qpotential_system( pdb.topology, Rc=cutoff ), 'npt': nptOn,
        'run': runSystems, 'color': 'black', 'label': '$q$ ($n=\infty$)', 'ls':'-', 'alpha':0.8, 'plot':False, 'linewidth':1
    })
])

In [ ]:
%cd $tempdir

Here we perform the actual MD simulation, incl. minimization of all defined systems

In [ ]:
%cd $tempdir

for name, prop in systemlist.items(): # loop over systems
    
    if prop['run']:

        print(name)

        if prop['npt']: # NVT -> NPT ensemble ?
            print('    adding barostat.')
            barostat = MonteCarloBarostat(1.0*bar, 298.15*kelvin, 25) 
            prop['system'].addForce(barostat)

        integrator = LangevinIntegrator( 298.15*kelvin, 1.0/picoseconds, 2*femtoseconds )
        integrator.setConstraintTolerance(0.00001)
        _platform = Platform.getPlatformByName( platform ) # CPU, OpenCL
        #_properties = {'CudaDeviceIndex': '0', 'CudaPrecision': 'mixed'}
        _properties = {'OpenCLDeviceIndex': '0', 'OpenCLPrecision': 'mixed'}
        sim = Simulation(pdb.topology, prop['system'], integrator, _platform, _properties )
        sim.context.setPositions(pdb.positions) # set particle positions

        if os.path.isfile( name+'.chk' ):
            with open( name+'.chk', 'rb') as f:
                print('    loading restart file.')
                sim.context.loadCheckpoint( f.read() )
        else:
            print('    minimizing energy...')
            sim.reporters.clear()
            %time sim.minimizeEnergy( tolerance=50*kilojoule/mole, maxIterations=1000 )
            sim.context.setVelocitiesToTemperature( 298.15*kelvin ) # initial random velocities

        print('    running Production...')
        sim.reporters.clear()
        sim.reporters.append( DCDReporter( name+'.dcd', 1000) )

        sim.reporters.append( StateDataReporter(name+'.energy', 1000, step=True, potentialEnergy=True,
                                                temperature=True, density=True) )

        sim.reporters.append( StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,
                                                temperature=True, density=True, separator='\t',
                                                progress=True,
                                                totalSteps = nsteps) )

        %time sim.step( nsteps )

        with open( name+'.chk', 'wb') as f:
            print('    saving restart file.')
            f.write( sim.context.createCheckpoint() )

        # save final configuration to PDB file
        # todo: something is fishy here... water molecules are outside the box
        #       in pdb although the trajectory looks fine. Å->nm problem?
        positions = sim.context.getState( getPositions=True ).getPositions()
        PDBFile.writeFile(sim.topology, positions, open( name+'.pdb', 'w'))
        print()
print('done.')

### Oxygen-oxygen radial distribution function

In [ ]:
plt.rcParams.update({'figure.figsize': [3.513475, 2.713475]})
lwA = 1.2;

import mdtraj as md

for name, prop in systemlist.items():
    if prop['plot']:
        print(name)

        rdffile = name+'OO.rdf'
        if os.path.isfile( rdffile ):
            rdf = np.loadtxt( rdffile )
        else:
            if os.path.isfile(name+'.dcd'):
                traj = md.load(name+'.dcd', top=name+'.pdb')
                sel = traj.top.select('name O')
                OOpairs = traj.top.select_pairs('name O', 'name O')
                rdf = md.compute_rdf( traj, pairs=OOpairs, bin_width=0.005, r_range=[0.2, boxlen/2/10] )
                np.savetxt( rdffile, rdf )
        
        if prop['label']=='No':
            plt.plot( rdf[0], rdf[1], color=prop['color'],ls=prop['ls'], alpha=prop['alpha'])
        else:
            plt.plot( rdf[0], rdf[1], label=prop['label'], color=prop['color'],ls=prop['ls'], alpha=prop['alpha'])

plt.xlim(0.1, 1.9)
plt.ylim(0.0,3.5)
plt.xlabel(r'$r$ / nm')
plt.ylabel(r'$g_{OO}(r)$')
plt.title(r'Cutoff '+str(cutoff))
plt.legend(loc=(1,0), frameon=False,ncol=1, labelspacing=1.75)
plt.savefig('spce-gofr.pdf', bbox_inches='tight')

In [ ]:
import mdtraj as md

erdf  = np.loadtxt('ewaldOO.rdf')

for name, prop in systemlist.items():
    if prop['plot']:
        if name!='qpot1':
            print(name)

            rdffile = name+'OO.rdf'
            if os.path.isfile( rdffile ):
                rdf = np.loadtxt( rdffile )
            else:
                if os.path.isfile(name+'.dcd'):
                    traj = md.load(name+'.dcd', top=name+'.pdb')
                    sel = traj.top.select('name O')
                    OOpairs = traj.top.select_pairs('name O', 'name O')
                    rdf = md.compute_rdf( traj, pairs=OOpairs, bin_width=0.005, r_range=[0.2, boxlen/2/10] )
                    np.savetxt( rdffile, rdf )
  
            if prop['label']=='No':
                plt.plot( rdf[0], np.log( rdf[1]*(erdf[1]**(-1)) ),lw=1.1,color=prop['color'],ls=prop['ls'], alpha=prop['alpha'])
            else:
                plt.plot( rdf[0], np.log( rdf[1]*(erdf[1]**(-1)) ),lw=1.1, label=prop['label'],color=prop['color'],ls=prop['ls'], alpha=prop['alpha'])

plt.xlim(0.25, 1.5)
plt.ylim(-0.03,0.03)
plt.xlabel(r'$r$ / nm',fontsize=fontSizeLabel)
plt.ylabel(r'$\ln\left(\frac{g_{OO}(r)}{g_{OO}^{Ewald}(r)}\right)$',fontsize=fontSizeLabel)
plt.legend(loc=(1,0), frameon=False,ncol=1, labelspacing=1.75)
plt.savefig('spce-gofr_diff.pdf', bbox_inches='tight')

### Static dielectric constant

The exact formula for $\epsilon_r$ is not given in the `mdtraj` documentation, but can be found in the source code on github,

https://github.com/mdtraj/mdtraj/blob/master/mdtraj/geometry/thermodynamic_properties.py#L90

In [ ]:
def getChargeVector( force, stripunit=True ):
    ''' Extract particle charge array from force object if available
    
    Note that this function assumes that charge is located at parameter
    position 0 which is default in NonbondedForce while it can be anywhere
    in CustomNonbondedForce, following the order in which
    parameters were added in python/xml.
    '''
    if isinstance(force, (NonbondedForce, CustomNonbondedForce) ):
        if 'getParticleParameters' in dir(force):
            chargelist = []
            for i in range( force.getNumParticles() ):
                charge = force.getParticleParameters( i )[ 0 ]
                if stripunit:
                    if isinstance(charge, Quantity):
                        charge = charge / elementary_charge # strip unit
                chargelist.append( charge )
            return chargelist
    return None
    
el = 1.60217656535*(10^(-19))
nm = 1e-9
D = 0.020819434*nm*el
for name, prop in systemlist.items():

        for force in prop['system'].getForces():
            if os.path.isfile(name+'.dcd'):
                if isinstance(force, (NonbondedForce, CustomNonbondedForce) ):
                    charges = getChargeVector( force )
                    traj = md.load(name+'.dcd', top=name+'.pdb')

                    l = int(len(traj)*0.8) # Get length of simulation (remove equilibration)
                    startV = 1000 # Start after equilibration
                    
                    bins=10
                    dip_mom2 = [];
                    eps = np.arange(bins)
                    for k in range(bins):
                        tmp = md.dipole_moments(traj[int(startV+1+k*l/10):int(startV+l+k*l/10)], charges)
                        dipw = 2.351*D # Dipole moment of a spc/e water molecule
                        tmp2 = tmp*tmp*nm*el/dipw*nm*el/dipw/1000
                        dip2 = np.mean(tmp2,axis=0)

                        dip_mom2.append(np.mean(dip2) )
                        eps[k] = md.static_dielectric( traj[int(startV+1+k*l/10):int(startV+l+k*l/10)], charges, temperature=298.15)

                    print(name, ': diel. const.', np.average(eps), np.std(eps))

In [ ]:
t = pd.DataFrame(systemlist).T
t.diel.to_json('spce-dielectric.json')
t.diel

## Gromacs analysis

We now generate a topology (top/tpr) file for Gromacs in order to use use their analysis tools, in particular for dipol correlations. These files are identitical for all electrostatic schemes.
**NOTE**: Depending on the size of the water box, `grompp` may complain that the cut-off is too short. Fix this by increasing the verlet tolerance. It will not matter for the analysis, but consider making a slightly larger system. 

In [ ]:
%%bash -s "$workdir"

cp $1/water.mdp .
cp $1/water.pdb .
cp $1/water.gro .

source $HOME/opt/bin/GMXRC.bash
rm -fR 
echo -n "6\n" | gmx -quiet -nobackup pdb2gmx -f water.pdb -o water.gro -p water.top -n water.ndx -water spce
gmx -quiet -nobackup grompp -f water.mdp -c water.gro -p water.top -o water.tpr -maxwarn 10

### Convert trajectory to XTC format and calculate dipolar correlations

See description of the `-g` option in `gmx dipoles`, relating to http://manual.gromacs.org/programs/gmx-dipoles.html (Nymand/Linse).

In [ ]:
for name, prop in systemlist.items():

    if os.path.isfile(name+'.dcd'):

        print(name)

        if not os.path.isfile( name+'.gkr.xvg' ):
            traj = md.load(name+'.dcd', top=name+'.pdb')
            traj.save_xtc(name+'.xtc')

            !source /home/mikael/opt/bin/GMXRC.bash ; echo -n "0" | gmx -quiet -nobackup dipoles -f $name'.xtc' -s water.tpr -temp 298.15 -g $name'.gkr.xvg'

            print()
print('done.')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(8, 7))

for name, prop in systemlist.items():
    if prop['plot']:
        if name in ['qpot', 'qpot1', 'qpot2', 'qpot3', 'qpot5', 'qpot7', 'SP1', 'SP3', 'ewald']:
            if os.path.isfile(name+'.gkr.xvg'):
                r, G, cos, hOO, gOO, energy = np.loadtxt(name+'.gkr.xvg', skiprows=27, unpack=True)

                ax1.plot(r, G,   label=prop['label'], color=prop['color'],linewidth=prop['linewidth'] )
                ax1.set_xlabel('$r$ / nm')
                ax1.set_ylabel('$G_k(r)$')
                #ax1.set_ylim(2, 3)
                ax1.set_xlim(0.2, 3.0)

                ax2.plot(r, cos, label=prop['label'], color=prop['color'],linewidth=prop['linewidth'] )
                ax2.set_xlabel('$r$ / nm')
                ax2.set_ylabel('$< \mu(0)\cdot \mu(r) >$')
                ax2.set_xlim(0.2, 1.9)
                ax2.set_ylim(-0.05, 0.38)

                ax3.plot(r, hOO, label=prop['label'], color=prop['color'],linewidth=prop['linewidth'] )
                ax3.set_xlabel('$r$ / nm')
                ax3.set_ylabel('$h_{OO}(r)$')
                ax3.set_xlim(0.2, 1.9)

                ax4.plot(r, gOO, label=prop['label'], color=prop['color'],linewidth=prop['linewidth'] )
                ax4.set_xlabel('$r$ / nm')
                ax4.set_ylabel('$g_{OO}(r)$')
                ax4.set_xlim(0.2, 1.9)

                prop['Gk'] = dict(x=r, y=G, label=r'$G_k(r)$')
                prop['hOO'] = dict(x=r, y=hOO, label=r'$hOO(r)$')
                prop['gOO'] = dict(x=r, y=gOO, label=r'$gOO(r)$')
                prop['cosine'] = dict(x=r, y=cos, label=r'cosine something')
    
#ax2.legend(loc=0, frameon=False, fontsize='medium')
ax2.legend(loc=(1,0), frameon=False, fontsize='medium')
plt.tight_layout()
fig.suptitle(r'$R_C$ =  '+str(cutoff))
fig.subplots_adjust(top=0.93)
plt.savefig('spce-kirkwood.pdf', bbox_inches='tight')


In [ ]:
for name, prop in systemlist.items():
    if prop['plot']:
        print(prop['label'])
        if os.path.isfile(name+'.gkr.xvg'):
            r, G, cos, hOO, gOO, energy = np.loadtxt(name+'.gkr.xvg', skiprows=27, unpack=True)
            plt.plot(r, G, label=prop['label'], color=prop['color'],linewidth=prop['linewidth'],
                     ls=prop['ls'], alpha=prop['alpha'] )
plt.xlabel('$r$ / nm')
plt.ylabel('$G_k(r)$')
plt.ylim(2.2, 4)
plt.xlim(0.2, 3.0)
    
#plt.legend(loc=0, frameon=False, fontsize='small', ncol=2)
plt.legend(loc=(1,0), frameon=False, fontsize='medium')
plt.tight_layout()
#fig.suptitle(r'Cutoff '+str(cutoff))
#fig.subplots_adjust(top=0.93)
plt.savefig('spce-kirkwood.pdf', bbox_inches='tight')

In [ ]:
er, eG, ecos, ehOO, egOO, eenergy = np.loadtxt('ewald.gkr.xvg', skiprows=27, unpack=True)
ecos=ecos[2:300]

print(er[1]-er[0])

for name, prop in systemlist.items():
    if prop['plot']:
        if (name in ['qpot2', 'qpot3', 'qpot5', 'qpot7', 'qpot', 'SP3', 'SP1']):

            if os.path.isfile(name+'.gkr.xvg'):
                r, G, cos, hOO, gOO, energy = np.loadtxt(name+'.gkr.xvg', skiprows=27, unpack=True)
                cos = cos[2:300]
                #plt.plot(r[2:300], ecos-cos, label=prop['label'], color=prop['color'], lw=2 )
                plt.plot(r[2:300], cos-ecos, label=prop['label'], color=prop['color'],linewidth=prop['linewidth'],ls=prop['ls'], alpha=prop['alpha'])
 
plt.xlim(0.2,1.9)
plt.ylim(-0.005,0.005)
#plt.legend(loc=0, frameon=False)
plt.xlabel('$r$ / nm')
plt.ylabel(r'$\langle \cos( \theta(r) ) \rangle_{\Omega} $')
plt.legend(loc=(1,0), frameon=False, fontsize='medium')

In [ ]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(systemlist, f, protocol=pickle.HIGHEST_PROTOCOL)  

In [ ]:
print('done')